### Task 1

<span style="color: green; font-size: 14px;">
Давай начнем с самого простого. В этом упражнении тебе необходимо получить отфильтрованные данные из таблицы 
в базе данных. Почему важно фильтровать данные именно в запросе, а не после этого в библиотеке Pandas?
Потому что таблицы могут иметь огромный размер. Если ты попыташься получить всю таблицу, то не сможешь ее "переварить" 
– у тебя может не хватить оперативной памяти. Всегда помни об этом.

Первый способ фильтрации — выбрать только те столбцы, которые тебе действительно нужны. 
Второй способ — выбрать только те строки, которые тебе действительно нужны.

Подробное описание задания:
</span>
<span style="color: green; font-size: 13px;">

1. Помести базу данных в подкаталог data в корневом каталоге, используемом в рамках этого проекта.
</span>

In [1]:
import os
import shutil

# создаём папку data, если она не существует
if not os.path.exists('data'):
    os.makedirs('data')

# копируем базу данных в папку data
shutil.copy2('../datasets/checking-logs.sqlite', 'data/checking-logs.sqlite')

'data/checking-logs.sqlite'

<span style="color: green; font-size: 13px;">

2. Создай соединение с базой данных с помощью библиотеки `sqlite3`.

3. Получи схему таблицы `pageviews` с помощью `pd.io.sql.read_sql` и запроса `PRAGMA table_info(pageviews);`.
</span>

In [2]:
import sqlite3
import pandas as pd

# создаём соединение
conn = sqlite3.connect('data/checking-logs.sqlite')

# получаем схему таблицы pageviews
schema = pd.io.sql.read_sql("PRAGMA table_info(pageviews);", conn)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,datetime,TIMESTAMP,0,None,0


<span style="color: green; font-size: 13px;">
    
4. Получи только первые `10` строк таблицы `pageviews`, чтобы проверить, как она выглядит.
</span>

In [3]:
# получаем первые 10 строк таблицы pageviews
pageviews = pd.io.sql.read_sql("SELECT * FROM pageviews LIMIT 10;", conn)
pageviews

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


<span style="color: green; font-size: 13px;">

5. Получи подтаблицу с помощью только **одного** запроса, в котором:
    - используются только `uid` и `datetime`;
    - используются только данные пользователей (`user_*`), и не используются данные администраторов;
    - сортировка выполняется по `uid` в порядке возрастания;
    - столбец индекса — это `datetime`;
    - `datetime` преобразован в `DatetimeIndex`;
    - имя датафрейма — `pageviews`.
</span>

In [4]:
# получаем подтаблицу
query = "SELECT uid, datetime FROM pageviews WHERE uid LIKE 'user_%' ORDER BY uid ASC;"
pageviews = pd.io.sql.read_sql(query, conn, index_col='datetime', parse_dates=['datetime'])
pageviews

,uid
datetime,
2020-04-26 21:53:59.624136,user_1
2020-04-26 22:06:19.478143,user_1
2020-04-26 22:12:09.614497,user_1
2020-04-30 19:29:01.831635,user_1
2020-05-05 20:26:32.894852,user_1
...,...
2020-04-29 16:51:21.877630,user_30
2020-05-09 20:30:47.034282,user_30
2020-05-22 11:30:18.368990,user_5


In [5]:
# получаем подтаблицу
query = "SELECT uid, datetime FROM pageviews WHERE uid LIKE 'user_%' ORDER BY CAST(SUBSTR(uid, 6) AS INTEGER) ASC;"
pageviews = pd.io.sql.read_sql(query, conn, index_col='datetime', parse_dates=['datetime'])
pageviews

,uid
datetime,
2020-04-26 21:53:59.624136,user_1
2020-04-26 22:06:19.478143,user_1
2020-04-26 22:12:09.614497,user_1
2020-04-30 19:29:01.831635,user_1
2020-05-05 20:26:32.894852,user_1
...,...
2020-05-21 01:46:08.730098,user_28
2020-05-21 18:45:20.441142,user_28
2020-04-17 22:46:26.785035,user_30


<span style="color: green; font-size: 13px;">
6. Закрой соединение с базой данных.
</span>

In [6]:
# закрываем соединение
conn.close()